In [31]:
import json
import shutil
import os
import requests

In [5]:
api_url = 'https://mtf-sat.synvinkel.org/timeseries?apikey=biscayabukten&fbclid=IwAR0woGRqFpiBML0h03XRs9Uz5ag3Vw834foaHnW4oCyRbRxjVyU8mmKOILE'
paramters  =' &lng=106.18&lat=53.98&startDate=2018-01-01&endDate=2018-12-30'

In [27]:
#get list of images for coordinates
req_data = requests.get(api_url + paramters).content
json_data = json.loads(req_data)
json_data

{'location': {'lng': '106.18', 'lat': '53.98'},
 'images': [{'bands': {'B1': 8051.92,
    'B10': 1245.37,
    'B11': 2973.79,
    'B12': 2957.75,
    'B2': 8120.11,
    'B3': 7280.03,
    'B4': 8518.78,
    'B5': 8996.58,
    'B6': 9306.95,
    'B7': 9421.15,
    'B8': 9227.95,
    'B8A': 9529.03,
    'B9': 6016.1},
   'cloudcover': 61.3645,
   'date': '2018-01-06',
   'time': 1515211892460,
   'url': 'https://mtf-sat.synvinkel.org/image/106.18/53.98/20180106T041129_20180106T041132_T48UWE-7d0.png',
   'rawUrl': 'https://mtf-sat.synvinkel.org/image/106.18/53.98/20180106T041129_20180106T041132_T48UWE-7d0.zip'},
  {'bands': {'B1': 5619.76,
    'B10': 226.91,
    'B11': 1322.25,
    'B12': 1244.26,
    'B2': 5136.05,
    'B3': 4267.59,
    'B4': 4510.3,
    'B5': 4593.2,
    'B6': 4654.7,
    'B7': 4675.18,
    'B8': 4363.92,
    'B8A': 4607.69,
    'B9': 2017.45},
   'cloudcover': 99.9999,
   'date': '2018-01-11',
   'time': 1515643885460,
   'url': 'https://mtf-sat.synvinkel.org/image/10

In [49]:
def toString(long, lat):
    return "lng:" + long + "lat:" + lat

def download_files(rel_dir, json_req):
    if os.path.exists(rel_dir):
        shutil.rmtree(rel_dir)
    os.makedirs(rel_dir)

    name = toString(json_data["location"]["lng"], json_data["location"]["lat"])
    
    for image in json_data["images"]:
        response = requests.get(image["url"], stream=True)
        file_name = rel_dir + name + "time:" + str(image["time"]) + ".png"
        with open(file_name, 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)
        del response

In [ ]:
download_files("./downloads/", json_data)

## Generate versioncontrollable artifacts

In [ ]:
!jupyter nbconvert --output-dir='./target' --to python TrailerGeneration.ipynb

In [ ]:
!jupyter nbconvert --output-dir='./target' --to html TrailerGeneration.ipynb